In [ ]:
vae_checkpoint="runs/vae_h100_full_power"

# Get User Latent Space

In [55]:
id2item_df = pd.read_csv(f"{vae_checkpoint}/unique_sid.txt", header=None).rename(columns={0: "ItemId"}).reset_index()

In [56]:
test_tr = df.copy(deep=True)
test_tr = test_tr.merge(id2item_df, how="inner", on="ItemId").rename(columns={"index": "sid"})
user_test = pd.DataFrame(test_tr["UserId"].unique(), columns=["UserId"]).reset_index().rename(columns={"index" : "uid"})
test_tr = pd.merge(test_tr, user_test, on="UserId")
test_tr[["uid", "sid"]].to_csv(f"{vae_checkpoint}/userset_latent.csv", index=False)

In [57]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]= "0,1"

In [ ]:
!python recvae/infer.py --return-latent --hidden-dim 3072 --latent-dim 2048 --dataset $vae_checkpoint --infer_data $vae_checkpoint/userset_latent.csv --model_path $vae_checkpoint/model.pt 

In [60]:
import pickle
with open(f"{vae_checkpoint}/result_csp.pkl", "rb") as f:
    result = pickle.load(f)

id2profile = dict(user_test.values)
profile2id = {value: key for key, value in id2profile.items()}
id2item = dict(id2item_df.values)
item2id = {value: key for key, value in id2item.items()}

In [83]:
from tqdm.notebook import tqdm
full_score = {}
for index, matrix in tqdm(enumerate(result), total=len(result)):
    user_item = {}
    for i in (np.argsort(matrix)[::-1][:20]):
        user_item[id2item[i]] = float(matrix[i])

    full_score[id2profile[index]] = user_item
    

  0%|          | 0/36748 [00:00<?, ?it/s]

In [84]:
import pickle
with open(f"{vae_checkpoint}/user_item_score.pkl", "wb") as f:
    pickle.dump(full_score, f)

In [ ]:
import pickle
with open(f"{vae_checkpoint}/result_latent.pkl", "rb") as f:
    result = pickle.load(f)

id2profile = dict(user_test.values)
profile2id = {value: key for key, value in id2profile.items()}

In [ ]:
import pickle
emb_user = dict([(k, v) for k, v in zip(profile2id.keys(), result)])
with open(f"{vae_checkpoint}/user_embedding.pkl", "wb") as f:
    pickle.dump(emb_user, f)

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]= "1"

# from sklearn.manifold import TSNE
from cuml import TSNE

tsne = TSNE(n_components=2, random_state=42, n_neighbors=90)
X_tsne = tsne.fit_transform(result)
tsne.kl_divergence_

In [ ]:
import plotly.express as px

fig = px.scatter(x=X_tsne[:, 0], y=X_tsne[:, 1], color = soft_clusters.argmax(axis=1)
                # color=X_tsne[:, 2]
                )
fig.update_layout(
    title="User Latent Space",
    xaxis_title="x",
    yaxis_title="y",
)
fig.show()

# Get Item Latent Space

In [ ]:
id2item_df = pd.read_csv(f"{vae_checkpoint}/unique_sid.txt", header=None).rename(columns={0: "ItemId"}).reset_index()

In [ ]:
id2item = dict(id2item_df.values)
item2id = {value: key for key, value in id2item.items()}

In [ ]:
id2item_df["uid"] = id2item_df["index"]
id2item_df["sid"] = id2item_df["index"]

In [ ]:
id2item_df[["uid", "sid"]].to_csv(f"{vae_checkpoint}/itemset_latent.csv", index=False)

In [ ]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"]= "0,1"

In [ ]:
!python recvae/infer.py --return-latent --hidden-dim 3072 --latent-dim 2048 --dataset $vae_checkpoint --infer_data $vae_checkpoint/itemset_latent.csv --model_path $vae_checkpoint/model.pt 

In [ ]:
import pickle
with open(f"{vae_checkpoint}/result_latent.pkl", "rb") as f:
    result = pickle.load(f)

In [ ]:
import pickle
emb_item = dict([(k, v) for k, v in zip(item2id.keys(), result)])
with open(f"{vae_checkpoint}/item_embedding.pkl", "wb") as f:
    pickle.dump(emb_item, f)

In [ ]:
from sklearn.manifold import TSNE
from cuml import TSNE

import os 
os.environ["CUDA_VISIBLE_DEVICES"]= "1"

tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(result)
tsne.kl_divergence_

In [ ]:
import plotly.express as px

fig = px.scatter(x=X_tsne[:, 0], y=X_tsne[:, 1])
fig.update_layout(
    title="Item Latent Space",
    xaxis_title="x",
    yaxis_title="y",
)
fig.show()